In [ ]:

import numpy as np
import propar
# Define the COM port and addresses of your instruments
COM_PORT = 'COM5'  # Replace with your actual COM port
BAUDRATE = 38400   # Replace with your instrument's baud rate
#Create list of adresse from 0 to 51
INSTRUMENT_ADDRESSES = [5,8]  # Replace with your instrument addresses

# Connect to the instruments
try:
    instruments = [
        propar.instrument(COM_PORT, address, baudrate=BAUDRATE)
        for address in INSTRUMENT_ADDRESSES
    ]
    print("Connected to instruments successfully!")
except Exception as e:
    print(f"Error connecting to instruments: {e}")
    exit(1)
# Read and display measurements


for instrument in instruments:
    try:
        # Read measurement parameter (adjust parameter numbers as needed)
        measurement = instrument.readParameter(205)  # Example parameter for "Measure"
        capacity_unit = instrument.readParameter(129)  # Example parameter for "Capacity Unit"
        if measurement is not None:
            print(f"Instrument {instrument.address}: Measurement = {measurement}, Unit = {capacity_unit}")
    except Exception as e:
        
        print(f"Error reading from instrument {instrument.address}: {e}")


Connected to instruments successfully!
Instrument 5: Measurement = 0.0, Unit = ln/min 
Instrument 8: Measurement = 0.0, Unit = ln/min 


In [8]:
def calculate_flows_with_constraints(C_tot_ppm, C1_ppm, C2_ppm, Q_max_individual=1.5):
    """
    Calculate the flow rates Q1 and Q2 required to achieve the desired methane concentration,
    respecting individual flow constraints.

    Parameters:
        C_tot_ppm (float): Desired methane concentration in the output (ppm).
        C1_ppm (float): Methane concentration in Gas1 (ppm).
        C2_ppm (float): Methane concentration in Gas2 (ppm).
        Q_max_individual (float): Maximum flow rate for each gas (ln/min).

    Returns:
        Q1 (float): Flow rate of Gas1 (ln/min), or None if no solution exists.
        Q2 (float): Flow rate of Gas2 (ln/min), or None if no solution exists.
    """
    # Convert ppm to fractional concentrations
    C_tot = C_tot_ppm / 1_000_000
    C1 = C1_ppm / 1_000_000
    C2 = C2_ppm / 1_000_000

    # Ensure inputs are valid
    if C1 == C2:
        raise ValueError("Methane concentrations in Gas1 and Gas2 must be different.")

    # Solve for Q1 and Q2
    Q1 = (C_tot - C2) / (C1 - C2)
    Q2 = 1 - Q1

    # Scale flows to total flow
    Q1 *= Q_max_individual
    Q2 *= Q_max_individual

    # Check constraints
    if Q1 > Q_max_individual or Q2 > Q_max_individual:
        # Scale flows proportionally if they exceed the maximum individual limit
        scaling_factor = min(Q_max_individual / max(Q1, Q2), 1.0)
        Q1 *= scaling_factor
        Q2 *= scaling_factor

        # Check if the scaled flows satisfy the concentration constraint
        C_actual = (Q1 * C1 + Q2 * C2) / (Q1 + Q2)
        if abs(C_actual - C_tot) > 1e-6:  # Allow a small tolerance
            return None, None  # No solution exists

    return Q1, Q2

# Example usage:
C_tot_ppm = 10000  # Desired methane concentration in the output (ppm)
C1_ppm = 200000  # Methane concentration in Gas1 (ppm)
C2_ppm = 0  # Methane concentration in Gas2 (ppm)

Q1, Q2 = calculate_flows_with_constraints(C_tot_ppm, C1_ppm, C2_ppm)
if Q1 is None or Q2 is None:
    print("No solution exists within the given constraints.")
else:
    print(f"Flow rate of Gas1 (Q1): {Q1:.2f} ln/min")
    print(f"Flow rate of Gas2 (Q2): {Q2:.2f} ln/min")


Flow rate of Gas1 (Q1): 0.07 ln/min
Flow rate of Gas2 (Q2): 1.42 ln/min


In [9]:
import tkinter as tk

root = tk.Tk()
root.title("Simulated RB-LCD-10B")
root.geometry("1280x800")  # Replace with the actual resolution
label = tk.Label(root, text="RB-LCD-10B Simulation", font=("Arial", 24))
label.pack(pady=20)
root.mainloop()


In [2]:
import propar
import time

class FlowController:
    def __init__(self, port='COM5', address=None):
        self.instrument = propar.instrument(port, address)
        self.setpoint_param = 9  # Parameter for setpoint
        self.measure_param = 8   # Parameter for measure
        
    def set_flow(self, percentage):
        """Set flow 0-100%"""
        try:
            if 0 <= percentage <= 100:
                value = int((percentage / 100.0) * 32000)
                self.instrument.writeParameter(self.setpoint_param, value)
                return True
            return False
        except Exception as e:
            print(f"Error setting flow: {e}")
            return False
            
    def read_flow(self):
        """Read current flow in %"""
        try:
            measure = self.instrument.readParameter(self.measure_param)
            return (measure / 32000.0) * 100
        except Exception as e:
            print(f"Error reading flow: {e}")
            return None

def main():
    # Create controller
    print("Connecting to flow controller...")
    controller = FlowController(address=5)
    
    while True:
        try:
            # Get command
            cmd = input("\nEnter flow setpoint (0-100%) or 'q' to quit: ").strip()
            
            # Check for quit
            if cmd.lower() == 'q':
                print("Stopping flow and exiting...")
                controller.set_flow(0)
                break
                
            # Try to set flow
            try:
                setpoint = float(cmd)
                if controller.set_flow(setpoint):
                    print(f"Setting flow to {setpoint}%")
                    # Read back actual flow
                    time.sleep(0.5)  # Wait for flow to stabilize
                    actual = controller.read_flow()
                    if actual is not None:
                        print(f"Current flow: {actual:.1f}%")
                else:
                    print("Invalid setpoint (use 0-100)")
            except ValueError:
                print("Invalid input - enter number between 0-100")
                
        except KeyboardInterrupt:
            print("\nStopping flow and exiting...")
            controller.set_flow(0)
            break

if __name__ == "__main__":
    main()

Connecting to flow controller...
Invalid input - enter number between 0-100
Setting flow to 43.0%
Current flow: 0.0%


: 

In [7]:
import propar
import time
import random 

dut = propar.instrument('com5')

print()
print("Testing using propar @", propar.__file__)
print()

n = 10

all_parameters = dut.db.get_all_parameters()
bt = time.perf_counter()
for i in range(n):
  for p in all_parameters:
    dut.read_parameters([p])  
et = time.perf_counter()

print("{:<20}{:>8}".format("read all parameters", (et - bt)                       / n))
print("{:<20}{:>8}".format("read one parameter ", (et - bt) / len(all_parameters) / n))


Testing using propar @ c:\Users\cruz\AppData\Local\anaconda3\envs\pylake\lib\site-packages\propar\__init__.py



KeyboardInterrupt: 